In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession
import numpy


In [2]:

sql_context = SparkSession.builder.master('local[*]').getOrCreate()



23/03/17 14:19:12 WARN Utils: Your hostname, siri-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.7 instead (on interface wlo1)
23/03/17 14:19:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/17 14:19:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = sql_context.read.csv("sangam.csv",header=True,inferSchema='True')


In [4]:
from pyspark.ml.feature import StringIndexer

ind = StringIndexer(inputCol = 'Status', outputCol = 'Status_index')
data=ind.fit(data).transform(data)



In [5]:
from pyspark.ml.feature import VectorAssembler

numericCols = ['DO', 'pH', 'ORP', 'Cond','Temp','WQI']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")

data = assembler.transform(data)




In [6]:
final=data.select("features","Status_index")
train, test = final.randomSplit([0.6, 0.4], seed = 2018)



In [7]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'Status_index',maxIter=50, regParam=0.01, elasticNetParam=0.0)



In [11]:
lrmodel = lr.fit(train)
# lrmodel.save("lr_model")

In [ ]:
lrmodel.save('./lrmodel')

In [12]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'Status_index')
rfmodel = rf.fit(train)